# Segmentation for Keyence

pycodex version 0.1.1

In [19]:
import sys

sys.path.append("/mnt/nfs/home/wenruiwu/projects")


In [26]:
import importlib
import os

from projects.pycodex.pycodex.data_process import crop
from projects.pycodex.pycodex.segmentation import io_keyence as io
from projects.pycodex.pycodex.segmentation import mobject as mo
from projects.pycodex.pycodex.segmentation import segmentation as seg
from projects.pycodex.pycodex.segmentation import utils as utl
from projects.pycodex.pycodex.segmentation import visualization as vis

for module in [crop, io, vis, seg, mo, utl]:
    importlib.reload(module)

from tifffile import tifffile

for module in [crop, io, vis, seg, mo, utl]:
    importlib.reload(module)

# GPU setup: Specify visible GPU(s) and allow memory growth
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

try:
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using GPU(s): {[gpu.name for gpu in gpus]}")
    else:
        print("No GPU detected, using CPU.")
except RuntimeError as e:
    print(f"GPU setup failed: {e}")


Using GPU(s): ['/physical_device:GPU:0']


# 00. Data Overview

In [21]:
final_dir = "/mnt/nfs/storage/RCC/RCC_formal_CODEX/RCC_TMA543/images/final"
metadata_dict = io.organize_metadata(final_dir)

unique_markers, _, _, _ = mo.metadata_dict_summary_marker(metadata_dict)
utl.display_items(unique_markers)
utl.display_items(list(metadata_dict.keys()))


Summary of Markers:
- Total unique markers: 77
- Unique markers: 75 ['ATP5A', 'C1Q', 'CA9_250ms', 'CA9_500ms', 'CD11b', 'CD11c', 'CD138', 'CD16', 'CD163', 'CD20', 'CD28', 'CD31', 'CD3e', 'CD4', 'CD45', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD68', 'CD69', 'CD8', 'CD86', 'Ch1Cy1', 'Ch1Cy10', 'Ch1Cy11', 'Ch1Cy12', 'Ch1Cy13', 'Ch1Cy14', 'Ch1Cy15', 'Ch1Cy16', 'Ch1Cy17', 'Ch1Cy18', 'Ch1Cy19', 'Ch1Cy2', 'Ch1Cy20', 'Ch1Cy21', 'Ch1Cy22', 'Ch1Cy23', 'Ch1Cy24', 'Ch1Cy25', 'Ch1Cy28', 'Ch1Cy3', 'Ch1Cy30', 'Ch1Cy31', 'Ch1Cy4', 'Ch1Cy5', 'Ch1Cy6', 'Ch1Cy7', 'Ch1Cy8', 'Ch1Cy9', 'Cytokeratin', 'DIG_TREM2', 'FoxP3', 'G6PD', 'GLUT1', 'GranzymeB', 'HLA1', 'IDO-1', 'IFN-y', 'Ki-67', 'LAG-3', 'MPO', 'NaKATP', 'P53', 'PD-1', 'PDL1_100_500ms', 'PDL1_50_250ms', 'Podoplanin', 'T-bet', 'TCF1_7', 'Tim-3', 'Tox_Tox2', 'VDAC1', 'aSMA']
- Blank markers: 2 ['Blank', 'BLANK_500ms_1']
- Markers duplicated in some regions: 0 []
- Markers missing in some regions: 0 []


,1,2,3,4,5,6,7,8,9,10
0,ATP5A,C1Q,CA9_250ms,CA9_500ms,CD11b,CD11c,CD138,CD16,CD163,CD20
1,CD28,CD31,CD3e,CD4,CD45,CD45RA,CD45RO,CD56,CD57,CD68
2,CD69,CD8,CD86,Ch1Cy1,Ch1Cy10,Ch1Cy11,Ch1Cy12,Ch1Cy13,Ch1Cy14,Ch1Cy15
3,Ch1Cy16,Ch1Cy17,Ch1Cy18,Ch1Cy19,Ch1Cy2,Ch1Cy20,Ch1Cy21,Ch1Cy22,Ch1Cy23,Ch1Cy24
4,Ch1Cy25,Ch1Cy28,Ch1Cy3,Ch1Cy30,Ch1Cy31,Ch1Cy4,Ch1Cy5,Ch1Cy6,Ch1Cy7,Ch1Cy8
5,Ch1Cy9,Cytokeratin,DIG_TREM2,FoxP3,G6PD,GLUT1,GranzymeB,HLA1,IDO-1,IFN-y
6,Ki-67,LAG-3,MPO,NaKATP,P53,PD-1,PDL1_100_500ms,PDL1_50_250ms,Podoplanin,T-bet
7,TCF1_7,Tim-3,Tox_Tox2,VDAC1,aSMA,,,,,


,1,2,3,4,5,6,7,8,9,10
0,reg001,reg002,reg003,reg004,reg005,reg006,reg007,reg008,reg009,reg010
1,reg011,reg012,reg013,reg014,reg015,reg016,reg017,reg018,reg019,reg020
2,reg021,reg022,reg023,reg024,reg025,,,,,


# 02. Segmentation

In [28]:
# parameter ^ #####
boundary_markers = ["CD45", "NaKATP", "HLA1", "G6PD", "CD45RO", "CD45RA", "CD8", "CD20", "CD31", "CA9_500ms"]
internal_markers = ["Ch1Cy1", "VDAC1", "ATP5A", "aSMA"]
pixel_size_um = 377.5202 / 1000
scale = True
maxima_threshold = 0.075
interior_threshold = 0.20
# parameter $ #####

marker_list = boundary_markers + internal_markers

marker_dir = "/mnt/nfs/storage/RCC/RCC_formal_CODEX/RCC_TMA543/images/final"
metadata_dict = io.organize_metadata(marker_dir)
marker_object = mo.organize_marker_object(metadata_dict, marker_list)

  0%|          | 0/25 [00:00<?, ?it/s]

CD45
NaKATP
HLA1
G6PD
CD45RO
CD45RA
CD8
CD20
CD31
CA9_500ms
Ch1Cy1
VDAC1
ATP5A


  4%|▍         | 1/25 [00:02<01:05,  2.73s/it]

aSMA
CD45
NaKATP
HLA1
G6PD
CD45RO
CD45RA
CD8
CD20
CD31
CA9_500ms
Ch1Cy1
VDAC1
ATP5A
aSMA


  8%|▊         | 2/25 [00:05<01:00,  2.63s/it]

...

 36%|███▌      | 9/25 [00:47<01:24,  5.27s/it]

VDAC1
ATP5A
aSMA
CD45
NaKATP
HLA1
G6PD
CD45RO
CD45RA
CD8
CD20
CD31
CA9_500ms
Ch1Cy1
VDAC1
ATP5A
aSMA


 40%|████      | 10/25 [00:49<01:14,  4.94s/it]


CD45


KeyboardInterrupt: 

In [32]:
marker = "CD45"
metadata_df = metadata_dict["reg001"]

metadata_df["path"][metadata_df["marker"] == marker].item()

'/mnt/nfs/storage/RCC/RCC_formal_CODEX/RCC_TMA543/images/final/reg001/reg001_cyc020_ch003_CD45.tif'

In [ ]:
segmentation_dir = "/mnt/nfs/home/wenruiwu/projects/shuli_rcc/data/output/segmentation_20241221"
mo.marker_object_segmentation_mesmer(
    segmentation_dir,
    marker_object,
    boundary_markers,
    internal_markers,
    pixel_size_um,
    scale,
    maxima_threshold,
    interior_threshold,
)
mo.extract_cell_features(marker_dir, segmentation_dir)
